In [1]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [2]:
digits = load_digits()
x = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.3)

In [3]:
print(x.shape,y.shape,x_train.shape)

(1797, 64) (1797, 10) (1257, 64)


## Dropout
> for tensorflow  Dropout mean that outputs the input element **scaled up by 1/keep_prob**, insteed of scaled down by keep_prob 

In [4]:
def add_layer(inputs,in_size,out_size,layer_name,
              activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size,out_size]))
    biases  = tf.Variable(tf.zeros([1,out_size])+0.1,)
    Wx_plus_b = tf.add(tf.matmul(inputs,Weights),biases)
    
    Wx_plus_b = tf.nn.dropout(Wx_plus_b,keep_prob)
    if activation_function== None:
        outputs=Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b,)
    tf.summary.histogram(layer_name+'/outputs',outputs)
    return outputs

In [5]:
keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32,[None,64])
ys = tf.placeholder(tf.float32,[None,10])

In [6]:
L1=add_layer(xs,64,50,layer_name='L1',
             activation_function=tf.nn.tanh)

In [7]:
prediction = add_layer(L1,50,10,layer_name='L2',
                       activation_function=tf.nn.softmax)

In [8]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),
                                             reduction_indices=[1]))
tf.summary.scalar('loss',cross_entropy)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [9]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [10]:
sess = tf.Session()

In [11]:
merged = tf.summary.merge_all()

In [19]:
train_writer = tf.summary.FileWriter('logd/train',sess.graph)
test_writer = tf.summary.FileWriter('logd/test',sess.graph)

In [20]:
init = tf.global_variables_initializer()
sess.run(init)

In [21]:
for i in range(500):
    sess.run(train_step,feed_dict={xs:x_train,ys:y_train,keep_prob:1})
    if i % 50 ==0:
        train_result = sess.run(merged,feed_dict={xs:x_train,ys:y_train,keep_prob:1})
        test_result = sess.run(merged,feed_dict={xs:x_test,ys:y_test,keep_prob:1})
        train_writer.add_summary(train_result,i)
        test_writer.add_summary(test_result,i)